<a href="https://colab.research.google.com/github/skywalker0803r/python-tutoring-materials/blob/main/%E8%AA%B2%E7%A8%8B/405_DQN_Reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 405 DQN Reinforcement Learning

View more, visit my tutorial page: https://mofanpy.com/tutorials/
My Youtube Channel: https://www.youtube.com/user/MorvanZhou
More about Reinforcement learning: https://mofanpy.com/tutorials/machine-learning/reinforcement-learning/

Dependencies:
* torch: 0.1.11
* gym: 0.8.1
* numpy

# 導入套件

In [117]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import gym

# 定義超參數

In [118]:
# Hyper Parameters
BATCH_SIZE = 32 #批量訓練的採樣數量
LR = 0.01 #學習率(太大不穩定,太小收斂慢)                  # learning rate
EPSILON = 0.9 #(有多少機率會選當前最優動作)               # greedy policy
GAMMA = 0.9 #獎勵衰減因子                # reward discount
TARGET_REPLACE_ITER = 100 #每隔多少次更新複製參數到target q 網路   # target update frequency
MEMORY_CAPACITY = 2000 # 資料庫容量大小(太小會導致資料量不足,太大導致資料更新很慢)
env = gym.make('CartPole-v0') #遊戲名稱
env = env.unwrapped # 後製處理
N_ACTIONS = env.action_space.n # 動作空間(ex: 上,下,左,右 則dim=4)
N_STATES = env.observation_space.shape[0] #狀態空間

In [119]:
N_ACTIONS,N_STATES

(2, 4)

# 定義神經網路 輸入:狀態 輸出:動作

In [120]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 10)
        self.fc1.weight.data.normal_(0, 0.1)   #權重初始化 initialization
        self.out = nn.Linear(10, N_ACTIONS)
        self.out.weight.data.normal_(0, 0.1)   #權重初始化 initialization

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value

In [121]:
class DQN(object):
    def __init__(self):
        # 估值網路 , 目標網路
        self.eval_net, self.target_net = Net(), Net()

        self.learn_step_counter = 0                                     # for target updating
        self.memory_counter = 0
        
        # 儲存遊戲經驗記憶庫memory                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory
        
        # 進行梯度下降的優化器
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        
        # 損失函數
        self.loss_func = nn.MSELoss()

    def choose_action(self, x):
      '''
      輸入狀態:x
      返回一個:action 動作
      '''
      x = Variable(torch.unsqueeze(torch.FloatTensor(x), 0))
      # input only one sample
      if np.random.uniform() < EPSILON:   # greedy
        actions_value = self.eval_net.forward(x)
        action = torch.max(actions_value, 1)[1].detach().numpy()[0]
      else:   # random#隨機選動作
        action = np.random.randint(0, N_ACTIONS)
      return action #整數標量

    # 如何儲存記憶到記憶庫的方式
    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

    # 模型做學習的方式  
    def learn(self):
        # target parameter update 目標網路的參數更新
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch transitions #採樣小批量的資料作訓練
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = Variable(torch.FloatTensor(b_memory[:, :N_STATES]))
        b_a = Variable(torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int)))
        b_r = Variable(torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2]))
        b_s_ = Variable(torch.FloatTensor(b_memory[:, -N_STATES:]))

        # q_eval w.r.t the action in experience
        # 用估值網路去計算 Q(s,a) 當前狀態下做特定動作的Q_value(q_eval)
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        # 用目標網路去計算 Q_(s_)可以得到的最大Q_value
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        # 目標Q值(q_target) = 獎勵 + GAMMA * (用目標網路去計算 Q_(s_)可以得到的最大Q_value)
        q_target = b_r + GAMMA * q_next.max(1)[0]   # shape (batch, 1)
        # 將q_eval和q_target去算LOSS值
        loss = self.loss_func(q_eval, q_target)

        # 根據LOSS值做梯度下降更新神經網路
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [122]:
dqn = DQN()

In [123]:
dqn

# 完整訓練流程

In [124]:

print('\nCollecting experience...')
# 迭代n次 這裡可以看他迭代400次
for i_episode in range(4000):
    # 環境初始化
    s = env.reset()
    # 這回合總分數初始化
    ep_r = 0
    # 遊戲開始
    while True:
        # 渲染畫面
        #env.render()

        # dqn算法根據state(s)選擇動作(action)(a)
        a = dqn.choose_action(s)

        # 執行 action 得到 next_state(s_),reward(r),done,info
        # 代表下一個狀態,獎勵,遊戲是否結束,其他資訊
        s_, r, terminated, truncated, info = env.step(a)

        # 如果時間到或是撞毀都算是遊戲結束
        if terminated or truncated:
          done = True
        else:
          done = False

        # modify(修改) the reward(獎勵) 非必要 (幫助快速收斂)
        x, x_dot, theta, theta_dot = s_
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2

        # 把(s(狀態), a(動作), r(獎勵), s_(下個狀態)) 儲存到我的遊戲記憶庫
        dqn.store_transition(s, a, r, s_)

        # 把整個回合的總獎勵加上 r
        ep_r += r
        # 當記憶庫更新的計數器達到了預定的計數
        if dqn.memory_counter > MEMORY_CAPACITY:
          # dqn做學習
          dqn.learn()
          # 如果遊戲結束,打印目前回合和當前回合分數
          if done:
            print('Ep: ', i_episode,'| Ep_r: ', round(ep_r, 2))
        # 如果遊戲結束break(跳脫迴圈重新開始下一輪遊戲)  
        if done:
            break
        # 把state(狀態)替換成next_state(下一偵狀態)
        s = s_


Ep:  202 | Ep_r:  1.26
Ep:  203 | Ep_r:  2.37
Ep:  204 | Ep_r:  3.91
Ep:  205 | Ep_r:  2.25
Ep:  206 | Ep_r:  3.67
Ep:  207 | Ep_r:  2.66
Ep:  208 | Ep_r:  2.92
Ep:  209 | Ep_r:  1.58
Ep:  210 | Ep_r:  4.25
Ep:  211 | Ep_r:  6.46
Ep:  212 | Ep_r:  6.09
Ep:  213 | Ep_r:  0.84
Ep:  214 | Ep_r:  1.15
Ep:  215 | Ep_r:  2.0
Ep:  216 | Ep_r:  3.2
Ep:  217 | Ep_r:  2.46
Ep:  218 | Ep_r:  0.91
Ep:  219 | Ep_r:  1.91
Ep:  220 | Ep_r:  3.6
Ep:  221 | Ep_r:  8.95
Ep:  222 | Ep_r:  3.04
Ep:  223 | Ep_r:  0.69
Ep:  224 | Ep_r:  6.73
Ep:  225 | Ep_r:  2.23
Ep:  226 | Ep_r:  1.45
Ep:  227 | Ep_r:  1.72
Ep:  228 | Ep_r:  3.38
Ep:  229 | Ep_r:  5.51
Ep:  230 | Ep_r:  3.2
Ep:  231 | Ep_r:  2.25
Ep:  232 | Ep_r:  3.91
Ep:  233 | Ep_r:  4.03
Ep:  234 | Ep_r:  0.76
Ep:  235 | Ep_r:  1.29
Ep:  236 | Ep_r:  1.15
Ep:  237 | Ep_r:  8.44
Ep:  238 | Ep_r:  2.38
Ep:  239 | Ep_r:  3.5
Ep:  240 | Ep_r:  3.23
Ep:  241 | Ep_r:  3.11
Ep:  242 | Ep_r:  2.77
Ep:  243 | Ep_r:  1.13
Ep:  244 | Ep_r:  6.86
Ep:  245 | Ep_r

KeyboardInterrupt: ignored